# Hello Radiomics example: using the feature extractor to calculate features

This example shows how to use the radiomics package and the feature extractor.
The feature extractor handles preprocessing, and then calls the needed featureclasses to calculate the features.
It is also possible to directly instantiate the feature classes. However, this is not recommended for use outside debugging or development. For more information, see `helloFeatureClass`.

In [1]:
import sys
stdout = sys.stdout  # This workaround is needed, as pykwalify reloads the sys package, resetting the stdout
stderr = sys.stderr

import os
import logging
from radiomics import featureextractor
import radiomics

sys.stdout = stdout  # Force the output back to the jupyter console
sys.stderr = stderr

## Setting up logging

In [2]:
logger = logging.getLogger('radiomics')  # Parent logger for the radiomics package

# Enable writing out the log using radiomics logger
radiomics.debug()  # Switch on radiomics logging from level=DEBUG (default level=WARNING)
# Alternative; set logging to a specific level
# logger.setLevel(logging.INFO)

# Prevent radiomics logger from printing out log entries with level < WARNING to the console
logger.handlers[0].setLevel(logging.WARNING)  # This limits the log messages printed to stdErr to level warning or higher
logger.propagate = False  # Do not pass log messages on to root logger

# Write out all log entries to a file, overwrite previous log
handler = logging.FileHandler(filename='testLog.txt', mode='w')
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # This determines how a log message should be formatted
handler.setFormatter(formatter)
logger.addHandler(handler)

## Getting the testcase

Testing data is contained in the pyradiomics/data folder, while this file is in the pyradiomics/bin/Notebooks folder. 

The next line of code gets the location of the current path and gets the location of the data as a relative path by going up two folders ("..") and then move into the data folders ("data").

For this to work, the current active directory should be pyradiomics/bin/notebooks, which is the case if this file is run from the pyradiomics/bin/Notebooks folder.

In [3]:
testCase = 'brain1'
dataDir = os.path.join(os.path.abspath(""), "..", "..", "data")
imageName = os.path.join(dataDir, testCase + '_image.nrrd')
maskName = os.path.join(dataDir, testCase + '_label.nrrd')

if not os.path.exists(imageName):
  print 'Error: problem finding input image', imageName
if not os.path.exists(maskName):
  print 'Error: problem finding input labelmap', maskName

## Initializing the feature extractor

#### Extraction Settings

In [4]:
kwargs = {}
kwargs['binWidth'] = 25
kwargs['resampledPixelSpacing'] = None
# kwargs['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
kwargs['interpolator'] = 'sitkBSpline'
kwargs['verbose'] = True

extractor = featureextractor.RadiomicsFeaturesExtractor(**kwargs)

#### Input images: applying filters

In [5]:
# By default, only 'original' (no filter applied) is enabled. Optionally enable some filters:

# extractor.enableInputImageByName('wavelet')
# extractor.enableInputImageByName('log', customArgs={'sigma':[3.0]})
# extractor.enableInputImageByName('square')
# extractor.enableInputImageByName('squareroot')
# extractor.enableInputImageByName('exponential')
# extractor.enableInputImageByName('logarithm')

# Alternative; set filters in one operation 
# This updates current enabled input images, i.e. overwrites custom settings. 
# However, input images already enabled, but not passed in this call, are not disabled.

# extractor.enableInputImages(wavelet={}, log={'sigma':[3.0]})

print "Enabled input images:"
for imageType in extractor.inputImages.keys():
    print '\t' + imageType

Enabled input images:
	original


#### Feature classes: setting which feature(classes) need to be calculated

In [6]:
# Disable all classes
extractor.disableAllFeatures()

# Enable all features in firstorder
extractor.enableFeatureClassByName('firstorder')

# Alternative; only enable 'Mean' and 'Skewness' features in firstorder
# extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])

## Getting the docstrings of the active features

In [7]:
print "Active features:"
for cls, features in extractor.enabledFeatures.iteritems():
  if len(features) == 0:
    features = extractor.getFeatureNames(cls)
  for f in features:
    print f
    print eval('extractor.featureClasses["%s"].get%sFeatureValue.__doc__' % (cls, f))

Active features:
10Percentile

    Calculate the 10\ :sup:`th` percentile in the image array.
    
90Percentile

    Calculate the 90\ :sup:`th` percentile in the image array.
    
Energy

    Calculate the Energy of the image array.

    :math:`energy = \displaystyle\sum^{N}_{i=1}{\textbf{X}(i)^2}`

    Energy is a measure of the magnitude of voxel values in
    an image. A larger values implies a greater sum of the
    squares of these values.
    
Entropy

    Calculate the Entropy of the image array.

    :math:`entropy = -\displaystyle\sum^{N_l}_{i=1}{p(i)\log_2\big(p(i)+\epsilon\big)}`

    Entropy specifies the uncertainty/randomness in the
    image values. It measures the average amount of
    information required to encode the image values.
    
InterquartileRange

    Calculate the interquartile range of the image array.

    :math:`interquartile\ range = \textbf{P}_{75} - \textbf{P}_{25}`, where :math:`\textbf{P}_{25}` and
    :math:`\textbf{P}_{75}` are the 25\ :sup:`th` a

## Calculating the values of the active features

In [8]:
print "Calculating features"
featureVector = extractor.execute(imageName, maskName)

for featureName in featureVector.keys():
  print "Computed %s: %s" % (featureName, featureVector[featureName])

Calculating features
		Computing firstorder
Computed general_info_BoundingBox: (162; 84; 11; 47; 70; 7)
Computed general_info_GeneralSettings: {'verbose': True; 'binWidth': 25; 'label': 1; 'interpolator': 'sitkBSpline'; 'resampledPixelSpacing': None; 'padDistance': 5}
Computed general_info_ImageHash: 5c9ce3ca174f0f8324aa4d277e0fef82dc5ac566
Computed general_info_ImageSpacing: (0.7812499999999999; 0.7812499999999999; 6.499999999999998)
Computed general_info_InputImages: {'original': {}}
Computed general_info_MaskHash: 9dc2c3137b31fd872997d92c9a92d5178126d9d3
Computed general_info_Version: v1.0.post11.dev0+g610dffc
Computed general_info_VolumeNum: 2
Computed general_info_VoxelNum: 4137
Computed original_firstorder_InterquartileRange: 253.0
Computed original_firstorder_Skewness: 0.275650859086
Computed original_firstorder_Uniformity: 0.0451569635559
Computed original_firstorder_MeanAbsoluteDeviation: 133.447261953
Computed original_firstorder_Energy: 33122817481.0
Computed original_firsto